## Código de Gray

In [2]:
function find_msb(n, size)
    for i in size - 1 : -1 : 0
        if n & (1 << i) != 0
            return i
        end
    end
    
    return 0
end

function is_bit_set(n, k) 
    return n & (1 << k) != 0
end
    
function set_bit(n, k)
    return n | (1 << k)
end

function binary_to_gray_repr(n)
    if n == 0
        return n
    end
    
    size = sizeof(n) * 8
        
    msb = find_msb(n, size)
    n_gray = n
    
    for i in size-2:-1:0
        if is_bit_set(n, i) != is_bit_set(n, i + 1)
            n_gray = set_bit(n_gray, i)
        else
            n_gray = n_gray & ~(1 << i)
        end
    end
    n_gray = set_bit(n_gray, msb)
    
    return n_gray
end;

In [3]:
bin = 0:255;
gray = binary_to_gray_repr.(bin);
bin2gray = Dict{UInt8, UInt8}(bin .=> gray);
gray2bin = Dict{UInt8, UInt8}(gray .=> bin);

## Problema

In [4]:
function example_program(x)
    return sin(π * x / 256)
end

example_program (generic function with 1 method)

## Seleção

In [40]:
function select(population, fitness)
    fitness_normalized = fitness ./ sum(fitness)
    
    fitness_cumulative = fitness_normalized
    for i in 2:length(population)
        fitness_cumulative[i] += fitness_cumulative[i - 1]
    end

    prob = rand(Float64, length(population))
    selected = [searchsortedfirst(fitness_cumulative, p) for p ∈ prob]
    
    return population[selected]    
end

select (generic function with 1 method)

## Crossover

In [307]:
function crossover(a, b)
    cromossome_size = sizeof(a) * 8
    # One-point crossover
    k = rand(0:cromossome_size - 1)
    mask = (2 ^ k) - 1
    
    return (mask & a) | (~mask & b)
end

function breed(parents, crossover_rate)
    offspring = copy(parents)
    
    for (i₁, i₂) ∈ zip(1:2:length(parents) - 1, 2:2:length(parents))
        parent₁ = rand(parents)
        parent₂ = rand(parents)
        
        offspring[i₁] = parent₁
        offspring[i₂] = parent₂
        
        if rand() > crossover_rate
            offspring[i₁] = crossover(parent₁, parent₂)
            offspring[i₂] = crossover(parent₁, parent₂)
        end
    end
    
    return offspring
end

breed (generic function with 1 method)

## Mutação

In [337]:
initial_population_size = 100
genome_size = 8
max_iterations = 10
crossover_rate = .2

0.2

In [338]:
population = binary_to_gray_repr.(rand(0:255, initial_population_size))

for i in 0:max_iterations
    # map(x -> gray2bin[x], population)
    fitness = example_program.(population)

    best_individual = argmax(fitness)
    fitnessₘₐₓ = fitness[best_individual]
    println("#$i Best fitness: $fitnessₘₐₓ\t$(gray2bin[population[best_individual]])")
    
    parents = select(population, fitness)
    population = breed(parents, crossover_rate)
end

#0 Best fitness: 0.9999247018391445	85
#1 Best fitness: 0.9999247018391445	85
#2 Best fitness: 0.9987954562051724	87
#3 Best fitness: 0.9981181129001492	82
#4 Best fitness: 0.9939069700023561	90
#5 Best fitness: 0.9987954562051724	87
#6 Best fitness: 0.9987954562051724	87
#7 Best fitness: 0.9981181129001492	82
#8 Best fitness: 0.9981181129001492	82
#9 Best fitness: 0.9981181129001492	82
#10 Best fitness: 0.9939069700023561	90
